In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

*Description rapide du projet* : 
- LA SBA c'est comme une assurance
- Elle ne prête pas l'argent directement à l'entreprise. Enfait une banque privée va prêter l'argent et la SBA va agir comme un garant. Si l'emprunteur ne paie plus, la SBA rembourse une partie de la perte à la banque.
- Si l'entreprise en question fait faillite, la banque perd donc de l'argent sur la partie non garantie. Elle doit donc trouver un équilibre entre accorder des prêts et gagner les intérêts et refuser des prêts trop risqués pour ne pas perdre de FP.

- **Notre projet vise donc à utiliser l'historique des données pour modéliser la proba qu'une entreprise fasse défaut et donc définir un seuil de décision pour la banque d'accorder ou non le prêt**

In [28]:
# 2000 - 2009
df1 = pd.read_csv('Brut_Data/foia-7a-fy2000-fy2009-asof-250930.csv')

# 2010 - 2019
df2 = pd.read_csv('Brut_Data/foia-7a-fy2010-fy2019-asof-250930.csv')

# 2020 - present
df3 = pd.read_csv('Brut_Data/foia-7a-fy2020-present-asof-250930.csv')

df_brut = pd.concat([df1, df2,  df3])

/var/folders/48/gm8qnf514r3bs39xcscgqymr0000gn/T/ipykernel_14650/3506059998.py:2: DtypeWarning: Columns (14,23,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Brut_Data/foia-7a-fy2000-fy2009-asof-250930.csv')
/var/folders/48/gm8qnf514r3bs39xcscgqymr0000gn/T/ipykernel_14650/3506059998.py:5: DtypeWarning: Columns (14,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Brut_Data/foia-7a-fy2010-fy2019-asof-250930.csv')
/var/folders/48/gm8qnf514r3bs39xcscgqymr0000gn/T/ipykernel_14650/3506059998.py:8: DtypeWarning: Columns (36,37,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('Brut_Data/foia-7a-fy2020-present-asof-250930.csv')


In [29]:
df_brut.shape

(1583598, 43)

In [30]:
df_brut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1583598 entries, 0 to 347513
Data columns (total 43 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   AsOfDate                     1583598 non-null  object 
 1   Program                      1583598 non-null  object 
 2   BorrName                     1583563 non-null  object 
 3   BorrStreet                   1583579 non-null  object 
 4   BorrCity                     1583598 non-null  object 
 5   BorrState                    1583598 non-null  object 
 6   BorrZip                      1583598 non-null  int64  
 7   LocationID                   1582421 non-null  float64
 8   BankName                     1583598 non-null  object 
 9   BankFDICNumber               1437677 non-null  float64
 10  BankNCUANumber               39120 non-null    float64
 11  BankStreet                   1582420 non-null  object 
 12  BankCity                     1582421 non-null  o

In [31]:
# GrossApproval correspond au montant total du prêt
# SBAGuaranteedApproval correspond au montant garantit SBA
pd.options.display.max_columns = None
df_brut.head(3)

,AsOfDate,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,LocationID,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFY,FirstDisbursementDate,ProcessingMethod,Subprogram,InitialInterestRate,FixedorVariableInterestRate,TerminMonths,NAICSCode,NAICSDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidinFullDate,ChargeoffDate,GrossChargeoffAmount,RevolverStatus,JobsSupported,CollateralInd,SoldSecondMarketInd
0,2025-09-30,7A,AUTOHAUS STUTTGART,7961 BALBOA AVENUE,SAN DIEGO,CA,92111,199911.0,"Wachovia SBA Lending, Inc.",NaN,NaN,1620 E. Roseville Pkwy,ROSEVILLE,CA,95661,914000.0,685500.0,10/01/1999,2000,NaN,Preferred Lenders Program,Guaranty,NaN,NaN,300,811111.0,General Automotive Repair,NaN,NaN,SAN DIEGO,CA,SAN DIEGO DISTRICT OFFICE,52.0,INDIVIDUAL,NaN,CANCLD,NaN,NaN,0.00,0,10.0,NaN,NaN
1,2025-09-30,7A,COTTMAN TRANSMISSION,4210 RED BLUFF RD,PASADENA,TX,77503,262382.0,Loans from Old Closed Lenders,NaN,NaN,409 3rd Street S.W.,Washington,DC,20416,460400.0,345300.0,10/01/1999,2000,10/31/1999,Preferred Lenders Program,Guaranty,NaN,NaN,217,811113.0,Automotive Transmission Repair,20550,COTTMAN TRANSMISSION,HARRIS,TX,HOUSTON DISTRICT OFFICE,29.0,CORPORATION,NaN,CHGOFF,NaN,06/05/2001,123184.59,0,12.0,Y,Y
2,2025-09-30,7A,GOSSELIN & SON TRUCKING INC,660 OLD FALL RIVER RD,NORTH DARTMOUTH,MA,2747,9551.0,"Bank of America, National Association",3510.0,NaN,100 North Tryon Street,Charlotte,NC,28255,36000.0,18000.0,10/01/1999,2000,10/31/1999,SBA Express Program,FA$TRK (Small Loan Express),NaN,NaN,36,NaN,NaN,NaN,NaN,BRISTOL,MA,MASSACHUSETTS DISTRICT OFFICE,9.0,CORPORATION,NaN,PIF,05/31/2005,NaN,0.00,0,2.0,NaN,NaN


In [32]:
df_brut.isna().sum()/df_brut.shape[0]

AsOfDate                       0.000000
Program                        0.000000
BorrName                       0.000022
BorrStreet                     0.000012
BorrCity                       0.000000
BorrState                      0.000000
BorrZip                        0.000000
LocationID                     0.000743
BankName                       0.000000
BankFDICNumber                 0.092145
BankNCUANumber                 0.975297
BankStreet                     0.000744
BankCity                       0.000743
BankState                      0.000746
BankZip                        0.000743
GrossApproval                  0.000000
SBAGuaranteedApproval          0.000000
ApprovalDate                   0.000000
ApprovalFY                     0.000000
FirstDisbursementDate          0.136391
ProcessingMethod               0.000000
Subprogram                     0.000000
InitialInterestRate            0.412798
FixedorVariableInterestRate    0.412797
TerminMonths                   0.000000


In [33]:
for col in df_brut.columns:
    if (df_brut[col].isna().sum()/df_brut.shape[0]) > 0.3:
        df_brut = df_brut.drop(col, axis= 1)
    else:
        pass

In [34]:
# On passe à 34 colonnes
df_brut.isna().sum()/df_brut.shape[0]

AsOfDate                 0.000000
Program                  0.000000
BorrName                 0.000022
BorrStreet               0.000012
BorrCity                 0.000000
BorrState                0.000000
BorrZip                  0.000000
LocationID               0.000743
BankName                 0.000000
BankFDICNumber           0.092145
BankStreet               0.000744
BankCity                 0.000743
BankState                0.000746
BankZip                  0.000743
GrossApproval            0.000000
SBAGuaranteedApproval    0.000000
ApprovalDate             0.000000
ApprovalFY               0.000000
FirstDisbursementDate    0.136391
ProcessingMethod         0.000000
Subprogram               0.000000
TerminMonths             0.000000
NAICSCode                0.012490
NAICSDescription         0.029182
ProjectCounty            0.000000
ProjectState             0.000000
SBADistrictOffice        0.000000
CongressionalDistrict    0.001516
BusinessType             0.000000
LoanStatus    

In [35]:
# faire gaffe à pas mettre cette variable dans le modèle
df_brut.loc[df_brut["GrossChargeoffAmount"]>0]["LoanStatus"].value_counts()

LoanStatus
CHGOFF    182040
Name: count, dtype: int64

In [36]:
# target 
df_brut['LoanStatus'].value_counts()

LoanStatus
PIF       896899
EXEMPT    290517
CANCLD    188919
CHGOFF    182061
COMMIT     25199
Name: count, dtype: int64

les modalités de la variable cible:
- **PIF** = PAID in full (donc remboursement total de l'entreprise et pas de défaut)
- **CHGOFF** = CHARGED OFF (donc l'emprunteur a arrêter de payer ==> Défaut)

ON a également:
- **COMMIT** = COMMITMENT / UNDISBURSED (à supprimer car le prêt n'a pas encore été donné)
- **CANCLD** = Cancelled (le contrat est annulé et le prêt n'a pas été approuvé donc à supprimer aussi de la base)

In [37]:
df_brut = df_brut.loc[(df_brut["LoanStatus"] == 'PIF') | (df_brut['LoanStatus'] == 'CHGOFF')]

In [38]:
df_brut["LoanStatus"].value_counts(normalize=True)
#c bon 

LoanStatus
PIF       0.831263
CHGOFF    0.168737
Name: proportion, dtype: float64

In [39]:
# Pour ne pas faire de data leakage et colonnes inutiles pour le moment
df_brut = df_brut.drop(columns={"GrossChargeoffAmount",
                               "AsOfDate"})

In [40]:
df_brut.columns

Index(['Program', 'BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip',
       'LocationID', 'BankName', 'BankFDICNumber', 'BankStreet', 'BankCity',
       'BankState', 'BankZip', 'GrossApproval', 'SBAGuaranteedApproval',
       'ApprovalDate', 'ApprovalFY', 'FirstDisbursementDate',
       'ProcessingMethod', 'Subprogram', 'TerminMonths', 'NAICSCode',
       'NAICSDescription', 'ProjectCounty', 'ProjectState',
       'SBADistrictOffice', 'CongressionalDistrict', 'BusinessType',
       'LoanStatus', 'RevolverStatus', 'JobsSupported', 'CollateralInd'],
      dtype='object')

In [41]:
# On binarise la cible
df_brut["LoanStatus"] = np.where(df_brut["LoanStatus"] == 'PIF', 0, 1)

In [42]:
df_brut["LoanStatus"].value_counts(normalize=True)

LoanStatus
0    0.831263
1    0.168737
Name: proportion, dtype: float64

In [43]:
df_brut.head()

,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,LocationID,BankName,BankFDICNumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFY,FirstDisbursementDate,ProcessingMethod,Subprogram,TerminMonths,NAICSCode,NAICSDescription,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,LoanStatus,RevolverStatus,JobsSupported,CollateralInd
1,7A,COTTMAN TRANSMISSION,4210 RED BLUFF RD,PASADENA,TX,77503,262382.0,Loans from Old Closed Lenders,NaN,409 3rd Street S.W.,Washington,DC,20416,460400.0,345300.0,10/01/1999,2000,10/31/1999,Preferred Lenders Program,Guaranty,217,811113.0,Automotive Transmission Repair,HARRIS,TX,HOUSTON DISTRICT OFFICE,29.0,CORPORATION,1,0,12.0,Y
2,7A,GOSSELIN & SON TRUCKING INC,660 OLD FALL RIVER RD,NORTH DARTMOUTH,MA,2747,9551.0,"Bank of America, National Association",3510.0,100 North Tryon Street,Charlotte,NC,28255,36000.0,18000.0,10/01/1999,2000,10/31/1999,SBA Express Program,FA$TRK (Small Loan Express),36,NaN,NaN,BRISTOL,MA,MASSACHUSETTS DISTRICT OFFICE,9.0,CORPORATION,0,0,2.0,NaN
3,7A,Vande Hey Company Inc.,N2093 COUNTY ROAD N,APPLETON,WI,54915,75939.0,"Associated Bank, National Association",5296.0,200 N Adams St,GREEN BAY,WI,54301,100000.0,80000.0,10/01/1999,2000,11/30/1999,7a General,Guaranty,120,NaN,NaN,OUTAGAMIE,WI,WISCONSIN DISTRICT OFFICE,6.0,CORPORATION,0,0,0.0,N
4,7A,NORTHSTAR ANIMAL CLINIC PC,2004 BLACKBURN RD.,SACHSE,TX,75048,71616.0,LegacyTexas Bank,18843.0,PO Box 869111,Plano,TX,75024,151000.0,113250.0,10/01/1999,2000,12/31/1999,Preferred Lenders Program,Guaranty,84,235410.0,Masonry and Stone Contractors,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,0,0,3.0,NaN
5,7A,HOLLY INN PARTNERSHIP,153 AMPEY RD.,PAW PAW,MI,49079,58036.0,Fifth Third Bank,6672.0,38 Fountain Sq Plz,CINCINNATI,OH,45263,653000.0,489750.0,10/01/1999,2000,10/31/2000,Preferred Lenders Program,Guaranty,180,551112.0,Offices of Other Holding Companies,VAN BUREN,MI,MICHIGAN DISTRICT OFFICE,6.0,PARTNERSHIP,0,0,23.0,NaN


In [44]:
df_brut.columns

Index(['Program', 'BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip',
       'LocationID', 'BankName', 'BankFDICNumber', 'BankStreet', 'BankCity',
       'BankState', 'BankZip', 'GrossApproval', 'SBAGuaranteedApproval',
       'ApprovalDate', 'ApprovalFY', 'FirstDisbursementDate',
       'ProcessingMethod', 'Subprogram', 'TerminMonths', 'NAICSCode',
       'NAICSDescription', 'ProjectCounty', 'ProjectState',
       'SBADistrictOffice', 'CongressionalDistrict', 'BusinessType',
       'LoanStatus', 'RevolverStatus', 'JobsSupported', 'CollateralInd'],
      dtype='object')

In [45]:
df_brut.RevolverStatus.value_counts()

RevolverStatus
0    746219
1    332741
Name: count, dtype: int64

In [46]:
df_brut.to_pickle("data_propre.pkl")